In [1]:
#中文填空


In [1]:
#中文分类
# 导入工具包
import os
import sys
import torch.nn as nn
# 添加环境变量
os.environ['TRANSFORMERS_OFFLINE'] = '0'
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
import time
from torch.utils.data import DataLoader
from tqdm import tqdm
#加载分词器
my_tokenizer = BertTokenizer.from_pretrained(r'D:\bert')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# 2 加载模型
my_model = BertModel.from_pretrained(r'D:\bert').to(device)#和自动模型区别只在于导包不相同

C:\Users\Lenovo\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


Some weights of the model checkpoint at D:\bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def collate_fn(data):
    #print(data)
    seq = [value['text'] for value in data]
    #进行编码
    input = my_tokenizer.batch_encode_plus(seq,padding = 'max_length',truncation=True,max_length=32,return_tensors='pt')
    input_ids = input['input_ids']
    token_type_ids = input['token_type_ids']
    attention_mask = input['attention_mask']
    #吧第十六个词变为mask
    labels = input_ids[:,16].reshape(-1).clone()
    input_ids[:,16] = my_tokenizer.get_vocab()[my_tokenizer.mask_token]
    labels = torch.tensor(labels,dtype = torch.long)
    #print(input)
    #print(labels)
    return input_ids,token_type_ids,attention_mask,labels

In [11]:
#定义测试函数  
def test_dataloader():
    train_dataset = load_dataset('csv',data_files = './train.csv',split = 'train')
    print(train_dataset)
    #对数据进行处理 只获取text大于32的文本
    new_train_dataset = train_dataset.filter(lambda x:len(x['text']) > 32)
    #print(new_train_dataset)
    #将上述数据进行封装
    train_dataloader = DataLoader(dataset = new_train_dataset,batch_size = 8,shuffle=True,collate_fn =collate_fn,drop_last=True)
    #遍历
    #for input_ids,token_type_ids,attention_mask,labels in train_dataloader:
        #print('test')
       # print(labels.shape)
        #brea
    return train_dataloader
test_dataloader()

Dataset({
    features: ['label', 'text'],
    num_rows: 9600
})


In [12]:
class Mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(768,my_tokenizer.vocab_size)#21128
    def forward(self,input_ids,token_type_ids,attention_mask):
        #预训练模型的参数不更新
        with torch.no_grad():
            output = my_model(input_ids = input_ids,
                              attention_mask = attention_mask,
                              token_type_ids = token_type_ids)
        #只去除16个张量
        out = self.fc(output['last_hidden_state'][:,16])
        return out#[8,21128]

#train_dataloader = test_dataloader()
#my_model = Mymodel().to(device)
#for input_ids,token_type_ids,attention_mask,labels in train_dataloader:
    #input_ids = input_ids.to(device)
    #token_type_ids = token_type_ids.to(device)
    #attention_mask = attention_mask.to(device)
    #output = my_model(input_ids,token_type_ids,attention_mask)
    #print(output.shape)
    #break

In [14]:
#模型训练
def train_model():
    #实例化数据源对象
    train_dataset = load_dataset('csv',data_files = './train.csv',split = 'train')
    train_dataset = train_dataset.filter(lambda x:len(x['text']) > 32)
    #将上述的数据进行封装
    train_dataloader = DataLoader(dataset =train_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #实例化模型
    mymodel = Mymodel().to(device)
    #实例化优化器
    my_optimizer = AdamW(mymodel.parameters(),lr = 5e-4)
    #实例化损失函数
    my_criterion = torch.nn.CrossEntropyLoss()
    # 不训练预训练模型 只让预训练模型计算数据特征 不需要计算梯度
    for param in my_model.parameters():
        param.requires_grad_(False)#删除的时候最原始的模型会进行更新
    #设置训练参数
    epochs = 3
    #设置训练模式
    mymodel.train()
    #开始训练
    for epoch in range(epochs):
        start_time = time.time()
        #开启内部数据迭代
        for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(train_dataloader),start = 1):
            #数据送入模型
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels_y = labels_y.to(device)
            output = mymodel(input_ids,token_type_ids,attention_mask)
            #print(output)
            #计算损失
            myloss = my_criterion(output,labels_y)
            #print(myloss)
            #梯度清0
            my_optimizer.zero_grad()
            #反向传播
            myloss.backward()
            #梯度更新
            my_optimizer.step()
            if idx % 20 == 0:
                out = output.argmax(dim=1) # [8,2] --> (8,)
                accuracy = (out == labels_y).sum().item() / len(labels_y)
                print(f'轮次:{epoch+1} 迭代数:{idx} 损失:{myloss.item():.6f} 准确率:{accuracy:.3f} 时间:{time.time()-start_time}s')

        # 每个轮次保存模型
        torch.save(mymodel.state_dict(), './dill_%d.bin' % (epoch + 1))

train_model()

  0%|          | 0/1129 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25140\4087393607.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)
  2%|▏         | 24/1129 [00:00<00:34, 32.16it/s]

轮次:1 迭代数:20 损失:9.511340 准确率:0.125 时间:0.6373248100280762s


  4%|▍         | 44/1129 [00:01<00:34, 31.85it/s]

轮次:1 迭代数:40 损失:9.082159 准确率:0.125 时间:1.2742724418640137s


  6%|▌         | 64/1129 [00:02<00:33, 32.13it/s]

...

 98%|█████████▊| 1104/1129 [00:34<00:00, 32.10it/s]

轮次:3 迭代数:1100 损失:1.201135 准确率:0.750 时间:34.44746446609497s


100%|█████████▉| 1124/1129 [00:35<00:00, 33.17it/s]

轮次:3 迭代数:1120 损失:0.170576 准确率:1.000 时间:35.05579972267151s


100%|██████████| 1129/1129 [00:35<00:00, 31.97it/s]


In [19]:
def test_model():
    #实例化数据源对象
    test_dataset = load_dataset('csv',data_files = './test.csv',split = 'train')
    test_dataset = test_dataset.filter(lambda x:len(x['text']) > 32)
    #将上述的数据进行封装
    test_dataloader = DataLoader(dataset =test_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #加载模型
    my_model = Mymodel().to(device)
    my_model.load_state_dict(torch.load('./dill_3.bin'))
    #定义测试超参数
    total = 0
    acc_num = 0
    my_model.eval()
    #开始测试
    for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(test_dataloader),start = 1):
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels_y = labels_y.to(device)
        with torch.no_grad():
            output = my_model(input_ids,token_type_ids,attention_mask)
        #计算正确的样本个数
        out = output.argmax(dim=1) # [8,2] --> (8,)
        acc_num = acc_num + (out == labels_y).sum().item() 
        total = total+len(labels_y)
        if idx %20 ==0:
            print(acc_num/total)
            print(idx+1,my_tokenizer.decode(input_ids[0]),end=' ')
            print('预测值 真实值:',my_tokenizer.decode(out[0]),my_tokenizer.decode(labels_y[0]))
test_model()

  0%|          | 0/141 [00:00<?, ?it/s]C:\Users\Lenovo\AppData\Local\Temp\ipykernel_25140\4087393607.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels,dtype = torch.long)
 20%|█▉        | 28/141 [00:00<00:02, 38.02it/s]

0.70625
21 [CLS] 实 在 看 不 懂 的 房 价 ， 房 间 较 干 净 ， [MASK] 是 象 是 临 时 房 一 样 ， 几 乎 没 有 隔 [SEP] 预测值 真实值: 就 但


 34%|███▍      | 48/141 [00:01<00:02, 37.65it/s]

0.696875
41 [CLS] 如 题.. 作 者 可 以 用 不 知 所 谓 来 形 [MASK]. 浪 费 了 我 的 时 间. 她 顶 多 也 就 [SEP] 预测值 真实值: 容 容


 48%|████▊     | 68/141 [00:01<00:01, 37.99it/s]

...

0.68625
101 [CLS] 书 里 基 本 都 是 日 本 的 生 活 环 境, 与 [MASK] 内 还 是 有 些 区 别 的 可 能 是 自 己 没 [SEP] 预测值 真实值: 国 国


 91%|█████████ | 128/141 [00:03<00:00, 37.55it/s]

0.69375
121 [CLS] 1. 虽 然 小 ， 但 是 我 这 个 大 手 用 起 [MASK] 还 行 ， 键 盘 的 手 感 可 以 媲 美 [UNK] 了 [SEP] 预测值 真实值: 来 来


100%|██████████| 141/141 [00:03<00:00, 37.33it/s]

0.69375
141 [CLS] 前 台 服 务 较 差 ， 不 为 客 户 着 想 。 房 [MASK] 有 朋 友 来 需 要 打 扫 ， 呼 叫 了 两 个 [SEP] 预测值 真实值: 间 间
